In [ ]:
# cmd shift 5 for area, this code for x_y
import pyautogui
import time

print("🖱️  Mouse position (1/sec) - Ctrl+C to quit")
try:
    while True:
        x, y = pyautogui.position()
        print(f"\rX: {x:4d} Y: {y:4d}", end='', flush=True)
        time.sleep(1)
except KeyboardInterrupt:
    print("\n✅ Stopped.")


🖱️  Mouse position (1/sec) - Ctrl+C to quit
X: 1147 Y:  326
✅ Stopped.


In [8]:
import pyautogui
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
from gradio_client import Client, handle_file
import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import datetime
import os

# CONFIGURATION
TABLE_X, TABLE_Y, TABLE_WIDTH, TABLE_HEIGHT = 50, 227, 283, 525
INTERVAL_SECONDS = 2.0
CSV_FILE = "fidelity_orderbook_log.csv"

# REUSE CLIENT (critical for speed)
client = Client("deepdoctection/deepdoctection")

print(f"🚀 Starting order book monitor (every {INTERVAL_SECONDS}s)")
print(f"Table region: ({TABLE_X}, {TABLE_Y}, {TABLE_WIDTH}, {TABLE_HEIGHT})")

iteration = 0
while True:
    start_time = time.time()
    iteration += 1

    try:
        # 1. CAPTURE SCREENSHOT
        screenshot = pyautogui.screenshot(region=(TABLE_X, TABLE_Y, TABLE_WIDTH, TABLE_HEIGHT))

        # 2. PREPROCESS IMAGE
        img = screenshot.convert("RGB")
        gray = ImageEnhance.Contrast(img.convert("L")).enhance(1.2)
        gray = gray.filter(ImageFilter.SHARPEN)
        inverted = ImageOps.invert(gray)
        binary = inverted.point(lambda p: 0 if p > 140 else 255)
        binary.save("processed_table_region.png")

        # 3. OCR VIA DEEPDOCTECTION
        result = client.predict(
            img=handle_file("processed_table_region.png"),
            pdf=None,
            max_datapoints=1,
            api_name="/analyze_image",
        )

        # 4. PARSE TABLE
        if isinstance(result, tuple) and len(result) >= 4:
            html_table = result[3]
            soup = BeautifulSoup(html_table, "html.parser")
            table = soup.find("table")
            df = pd.read_html(str(table))[0]

            # Ensure readable string headers (handle blanks)
            if df.columns.isnull().any():
                df.columns = [f"col_{i}" if pd.isna(c) else str(c).strip() for i, c in enumerate(df.columns)]
            else:
                df.columns = [str(c).strip() for c in df.columns]

            # 5. FLATTEN INTO SINGLE ROW SAFELY
            flat_values = df.to_numpy().ravel()
            col_labels = []
            for i in range(len(df)):
                for col in df.columns:
                    col_labels.append(f"{col}_{i+1}")

            # Handle mismatch gracefully
            if len(col_labels) != len(flat_values):
                min_len = min(len(col_labels), len(flat_values))
                col_labels = col_labels[:min_len]
                flat_values = flat_values[:min_len]

            # 6. BUILD ROW + SAVE (APPEND)
            row_data = [datetime.now().strftime("%Y-%m-%d %H:%M:%S")] + list(flat_values)
            col_names = ["timestamp"] + col_labels

            single_row = pd.DataFrame([row_data], columns=col_names)

            header_needed = not os.path.exists(CSV_FILE)
            single_row.to_csv(CSV_FILE, index=False, mode="a", header=header_needed)

            print(f"[{iteration:3d}] ✅ Snapshot saved | {df.shape[0]} rows → flattened to {single_row.shape[1]} columns")
            print(f"📁 Appended → {CSV_FILE}")
            print("-" * 60)

        else:
            print(f"[{iteration}] ❌ No table detected")

    except KeyboardInterrupt:
        print("\n🛑 Stopped by user")
        break
    except Exception as e:
        print(f"[{iteration}] ❌ Error: {e}")
        continue

    # Maintain consistent interval
    elapsed = time.time() - start_time
    sleep_time = max(0, INTERVAL_SECONDS - elapsed)
    if sleep_time > 0:
        time.sleep(sleep_time)
    else:
        print(f"[{iteration}] ⚡ Processed in {elapsed:.2f}s (faster than interval!)")

print(f"📊 Completed {iteration} iterations")


Loaded as API: https://deepdoctection-deepdoctection.hf.space ✔
🚀 Starting order book monitor (every 2.0s)
Table region: (50, 227, 283, 525)


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50643/3049008669.py:51: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


[  1] ✅ Snapshot saved | 26 rows → flattened to 157 columns
📁 Appended → fidelity_orderbook_log.csv
------------------------------------------------------------
[1] ⚡ Processed in 10.73s (faster than interval!)


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50643/3049008669.py:51: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


[  2] ✅ Snapshot saved | 26 rows → flattened to 157 columns
📁 Appended → fidelity_orderbook_log.csv
------------------------------------------------------------
[2] ⚡ Processed in 10.83s (faster than interval!)


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50643/3049008669.py:51: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


[  3] ✅ Snapshot saved | 26 rows → flattened to 157 columns
📁 Appended → fidelity_orderbook_log.csv
------------------------------------------------------------
[3] ⚡ Processed in 10.80s (faster than interval!)

🛑 Stopped by user
📊 Completed 4 iterations


In [9]:
import pandas as pd

# Read the CSV
df = pd.read_csv("fidelity_orderbook_log.csv")

# ALTERNATING HEADERS for bid/ask sides
BID_HEADERS = ['exch_b', 'size_b', 'bid', 'ask', 'size_a', 'exch_a']

# Keep timestamp + data columns (skip first 6 junk header columns)
data_columns = df.columns[6:]
df_data = df[['timestamp'] + list(data_columns)].copy()

# SHIFT LEFT: Remove first junk column and shift all data left by 1
first_junk_col = data_columns[0]
df_data = df_data.drop(columns=[first_junk_col])  # Drop first junk column

# Create PROPER alternating column names (now 1 less column)
col_names = ["timestamp"]
level = 1

for i in range(len(df_data.columns) - 1):  # Adjusted for dropped column
    base_header = BID_HEADERS[i % 6]
    col_names.append(f"{base_header}_{level}")
    
    if (i + 1) % 6 == 0:
        level += 1

# Assign the alternating column names
df_data.columns = col_names

# Save cleaned version
df_data.to_csv("fidelity_orderbook_clean.csv", index=False)

print("✅ SHIFTED LEFT (first 20):")
print(list(df_data.columns[:20]))
print("\nSample data:")
print(df_data[['timestamp', 'exch_b_1', 'size_b_1', 'bid_1', 'ask_1', 'size_a_1', 'exch_a_1']].head(2))


✅ SHIFTED LEFT (first 20):
['timestamp', 'exch_b_1', 'size_b_1', 'bid_1', 'ask_1', 'size_a_1', 'exch_a_1', 'exch_b_2', 'size_b_2', 'bid_2', 'ask_2', 'size_a_2', 'exch_a_2', 'exch_b_3', 'size_b_3', 'bid_3', 'ask_3', 'size_a_3', 'exch_a_3', 'exch_b_4']

Sample data:
             timestamp exch_b_1  size_b_1   bid_1   ask_1  size_a_1 exch_a_1
0  2025-12-06 09:33:24     ARCX         1  454.61  454.64       160     XNMS
1  2025-12-06 09:33:35     ARCX         1  454.61  454.64       160     XNMS


In [10]:
df_data

,timestamp,exch_b_1,size_b_1,bid_1,ask_1,size_a_1,exch_a_1,exch_b_2,size_b_2,bid_2,...,bid_24,ask_24,size_a_24,exch_a_24,exch_b_25,size_b_25,bid_25,ask_25,size_a_25,exch_a_25
0,2025-12-06 09:33:24,ARCX,1,454.61,454.64,160,XNMS,XNMS,80,454.68,...,453.42,455.09,460,ARCX,ARCX,1,453.31,455.1,316,ARCX
1,2025-12-06 09:33:35,ARCX,1,454.61,454.64,160,XNMS,XNMS,80,454.68,...,453.42,455.09,460,ARCX,ARCX,1,453.31,455.1,316,ARCX
2,2025-12-06 09:33:46,ARCX,1,454.61,454.64,160,XNMS,XNMS,80,454.68,...,453.42,455.09,460,ARCX,ARCX,1,453.31,455.1,316,ARCX


In [13]:
import pyautogui
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
from gradio_client import Client, handle_file
import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import datetime
import os

# CONFIGURATION
TABLE_X, TABLE_Y, TABLE_WIDTH, TABLE_HEIGHT = 50, 227, 283, 525
INTERVAL_SECONDS = 2.0
CSV_FILE = "fidelity_orderbook_log.csv"
CLEAN_CSV_FILE = "fidelity_orderbook_clean.csv"

# REUSE CLIENT (critical for speed)
client = Client("deepdoctection/deepdoctection")

print(f"🚀 Starting order book monitor (every {INTERVAL_SECONDS}s)")
print(f"Table region: ({TABLE_X}, {TABLE_Y}, {TABLE_WIDTH}, {TABLE_HEIGHT})")

iteration = 0
try:
    while True:
        start_time = time.time()
        iteration += 1

        try:
            # 1. CAPTURE SCREENSHOT
            screenshot = pyautogui.screenshot(region=(TABLE_X, TABLE_Y, TABLE_WIDTH, TABLE_HEIGHT))

            # 2. PREPROCESS IMAGE
            img = screenshot.convert("RGB")
            gray = ImageEnhance.Contrast(img.convert("L")).enhance(1.2)
            gray = gray.filter(ImageFilter.SHARPEN)
            inverted = ImageOps.invert(gray)
            binary = inverted.point(lambda p: 0 if p > 140 else 255)
            binary.save("processed_table_region.png")

            # 3. OCR VIA DEEPDOCTECTION
            result = client.predict(
                img=handle_file("processed_table_region.png"),
                pdf=None,
                max_datapoints=1,
                api_name="/analyze_image",
            )

            # 4. PARSE TABLE
            if isinstance(result, tuple) and len(result) >= 4:
                html_table = result[3]
                soup = BeautifulSoup(html_table, "html.parser")
                table = soup.find("table")
                df = pd.read_html(str(table))[0]

                if df.columns.isnull().any():
                    df.columns = [f"col_{i}" if pd.isna(c) else str(c).strip() for i, c in enumerate(df.columns)]
                else:
                    df.columns = [str(c).strip() for c in df.columns]

                flat_values = df.to_numpy().ravel()
                col_labels = []
                for i in range(len(df)):
                    for col in df.columns:
                        col_labels.append(f"{col}_{i+1}")

                if len(col_labels) != len(flat_values):
                    min_len = min(len(col_labels), len(flat_values))
                    col_labels = col_labels[:min_len]
                    flat_values = flat_values[:min_len]

                row_data = [datetime.now().strftime("%Y-%m-%d %H:%M:%S")] + list(flat_values)
                col_names = ["timestamp"] + col_labels

                single_row = pd.DataFrame([row_data], columns=col_names)

                header_needed = not os.path.exists(CSV_FILE)
                single_row.to_csv(CSV_FILE, index=False, mode="a", header=header_needed)

                print(f"[{iteration:3d}]  Snapshot saved | {df.shape[0]} rows → flattened to {single_row.shape[1]} columns")

            else:
                print(f"[{iteration}]  No table detected")

        except Exception as e:
            print(f"[{iteration}]  Error: {e}")
            continue

        elapsed = time.time() - start_time
        sleep_time = max(0, INTERVAL_SECONDS - elapsed)
        if sleep_time > 0:
            time.sleep(sleep_time)


except KeyboardInterrupt:
    print("\n Stopped by user — starting CSV cleanup...")

# ===============================
# CLEANUP AND REFORMAT STORED CSV
# ===============================
if os.path.exists(CSV_FILE):
    df = pd.read_csv(CSV_FILE)

    BID_HEADERS = ['exch_b', 'size_b', 'bid', 'ask', 'size_a', 'exch_a']

    # Keep timestamp + data columns (skip first 6 junk header columns)
    data_columns = df.columns[6:]
    df_data = df[['timestamp'] + list(data_columns)].copy()

    # Drop first junk column and shift left by one
    first_junk_col = data_columns[0]
    df_data = df_data.drop(columns=[first_junk_col])

    # Create proper alternating column names
    col_names = ["timestamp"]
    level = 1
    for i in range(len(df_data.columns) - 1):  # Adjust for dropped column
        base_header = BID_HEADERS[i % 6]
        col_names.append(f"{base_header}_{level}")
        if (i + 1) % 6 == 0:
            level += 1

    df_data.columns = col_names
    df_data.to_csv(CLEAN_CSV_FILE, index=False)

    print(f"✅ Cleaned CSV saved → {CLEAN_CSV_FILE}")
    print("First 20 columns:")
    print(list(df_data.columns[:20]))
    print("\nSample preview:")
    print(df_data[['timestamp', 'exch_b_1', 'size_b_1', 'bid_1', 'ask_1', 'size_a_1', 'exch_a_1']].head(2))
else:
    print("⚠️ No CSV file found; skipping cleanup.")


Loaded as API: https://deepdoctection-deepdoctection.hf.space ✔
🚀 Starting order book monitor (every 2.0s)
Table region: (50, 227, 283, 525)


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50643/3363499120.py:53: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


[  1] ✅ Snapshot saved | 26 rows → flattened to 157 columns
📁 Appended → fidelity_orderbook_log.csv
------------------------------------------------------------
[1] ⚡ Processed in 11.83s (faster than interval!)


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50643/3363499120.py:53: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


[  2] ✅ Snapshot saved | 26 rows → flattened to 157 columns
📁 Appended → fidelity_orderbook_log.csv
------------------------------------------------------------
[2] ⚡ Processed in 11.33s (faster than interval!)


...

[  5] ✅ Snapshot saved | 26 rows → flattened to 157 columns
📁 Appended → fidelity_orderbook_log.csv
------------------------------------------------------------
[5] ⚡ Processed in 11.48s (faster than interval!)


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50643/3363499120.py:53: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


[  6] ✅ Snapshot saved | 26 rows → flattened to 157 columns
📁 Appended → fidelity_orderbook_log.csv
------------------------------------------------------------
[6] ⚡ Processed in 11.26s (faster than interval!)


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50643/3363499120.py:53: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


[  7] ✅ Snapshot saved | 26 rows → flattened to 157 columns
📁 Appended → fidelity_orderbook_log.csv
------------------------------------------------------------
[7] ⚡ Processed in 10.75s (faster than interval!)

🛑 Stopped by user — starting CSV cleanup...
✅ Cleaned CSV saved → fidelity_orderbook_clean.csv
First 20 columns:
['timestamp', 'exch_b_1', 'size_b_1', 'bid_1', 'ask_1', 'size_a_1', 'exch_a_1', 'exch_b_2', 'size_b_2', 'bid_2', 'ask_2', 'size_a_2', 'exch_a_2', 'exch_b_3', 'size_b_3', 'bid_3', 'ask_3', 'size_a_3', 'exch_a_3', 'exch_b_4']

Sample preview:
             timestamp exch_b_1  size_b_1   bid_1   ask_1  size_a_1 exch_a_1
0  2025-12-06 09:35:31     ARCX         1  454.61  454.64       160     XNMS
1  2025-12-06 09:35:42     ARCX         1  454.61  454.64       160     XNMS


In [14]:
df_data

,timestamp,exch_b_1,size_b_1,bid_1,ask_1,size_a_1,exch_a_1,exch_b_2,size_b_2,bid_2,...,bid_24,ask_24,size_a_24,exch_a_24,exch_b_25,size_b_25,bid_25,ask_25,size_a_25,exch_a_25
0,2025-12-06 09:35:31,ARCX,1,454.61,454.64,160,XNMS,XNMS,80,454.68,...,453.42,455.09,460,ARCX,ARCX,1,453.31,455.1,316,ARCX
1,2025-12-06 09:35:42,ARCX,1,454.61,454.64,160,XNMS,XNMS,80,454.68,...,453.42,455.09,460,ARCX,ARCX,1,453.31,455.1,316,ARCX
2,2025-12-06 09:35:53,ARCX,1,454.61,454.64,160,XNMS,XNMS,80,454.68,...,453.42,455.09,460,ARCX,ARCX,1,453.31,455.1,316,ARCX
3,2025-12-06 09:36:03,ARCX,1,454.61,454.64,160,XNMS,XNMS,80,454.68,...,453.42,455.09,460,ARCX,ARCX,1,453.31,455.1,316,ARCX
4,2025-12-06 09:37:18,ARCX,1,454.61,454.64,160,XNMS,XNMS,80,454.68,...,453.42,455.09,460,ARCX,ARCX,1,453.31,455.1,316,ARCX
5,2025-12-06 09:37:29,ARCX,1,454.61,454.64,160,XNMS,XNMS,80,454.68,...,453.42,455.09,460,ARCX,ARCX,1,453.31,455.1,316,ARCX
6,2025-12-06 09:37:40,ARCX,1,454.61,454.64,160,XNMS,XNMS,80,454.68,...,453.42,455.09,460,ARCX,ARCX,1,453.31,455.1,316,ARCX
7,2025-12-06 09:37:51,ARCX,1,454.61,454.64,160,XNMS,XNMS,80,454.68,...,453.42,455.09,460,ARCX,ARCX,1,453.31,455.1,316,ARCX
8,2025-12-06 09:38:03,ARCX,1,454.61,454.64,160,XNMS,XNMS,80,454.68,...,453.42,455.09,460,ARCX,ARCX,1,453.31,455.1,316,ARCX
9,2025-12-06 09:38:14,ARCX,1,454.61,454.64,160,XNMS,XNMS,80,454.68,...,453.42,455.09,460,ARCX,ARCX,1,453.31,455.1,316,ARCX
